In [1]:
import awswrangler as awr
import pandas as pd
import os
import openpyxl


In [ ]:
#caminho_query = r"C:\Users\raphael.almeida\Documents\Processos\relatorio_inadimplencia\sql\faturas_mes.sql"
#with open(caminho_query,'r') as arquivo_query:
#    query = arquivo_query.read()
#df_mes = awr.athena.read_sql_query(query, database='silver')

#df_mes.drop_duplicates(subset='ponteiro', inplace=True)

In [6]:
caminho_xlsx = r"C:\Users\raphael.almeida\OneDrive - Grupo Unus\analise de dados - Arquivos em excel\Relatório de Inadimplência\relatorio_faturas_mes.xlsx"

df_mes_0 = pd.read_excel(caminho_xlsx)

In [7]:
# Filtrar linhas de julho de 2025 e somar os valores de 'valor_titulo' em df_mes_0
df_mes_0['data_vencimento'] = pd.to_datetime(df_mes_0['data_vencimento'])
linhas_julho_2025_0 = df_mes_0[
    (df_mes_0['data_vencimento'].dt.year == 2025) &
    (df_mes_0['data_vencimento'].dt.month == 7)
]
soma_valor_titulo_julho_2025_0 = linhas_julho_2025_0['valor_titulo'].sum()
print(f"Soma de valor_titulo das linhas de julho de 2025 em df_mes_0: {soma_valor_titulo_julho_2025_0}")


Soma de valor_titulo das linhas de julho de 2025 em df_mes_0: 48388134.449999996


In [ ]:
df_mes = df_mes_0

# Converter colunas de datas
df_mes['data_vencimento'] = pd.to_datetime(df_mes['data_vencimento'])
df_mes['data_reemissao'] = pd.to_datetime(df_mes['data_reemissao'], errors='coerce')

# Extrair ano e mês para comparação dentro do mesmo período
df_mes['ano_mes'] = df_mes['data_vencimento'].dt.to_period('M')

# Função que remove o original se houver duplicata reemitida no mesmo mês e com valor semelhante
def remover_duplicados(grupo):
    grupo = grupo.copy()
    for (mes), linhas_mes in grupo.groupby(['ano_mes']):
        for i, linha in linhas_mes.iterrows():
            similares = linhas_mes[
                (linhas_mes.index != i) &  # Não comparar com ele mesmo
                (abs(linhas_mes['valor_titulo'] - linha['valor_titulo']) <= 500) &  # Valor próximo
                (linhas_mes['data_vencimento'].dt.to_period('M') == linha['data_vencimento'].to_period('M'))  # Mesmo mês
            ]
            if not similares.empty:
                if linha['data_reemissao'] is pd.NaT:
                    grupo.drop(index=i, inplace=True)
    return grupo

# Aplicar por numero_documento e conjunto
df_mes_filtrado = df_mes.groupby(['numero_documento', 'conjunto'], group_keys=False).apply(remover_duplicados)

# Limpar colunas auxiliares
df_mes_filtrado = df_mes_filtrado.drop(columns=['ano_mes'])



C:\Users\raphael.almeida\AppData\Local\Temp\ipykernel_17224\917472037.py:24: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_mes_filtrado = df_mes.groupby(['numero_documento', 'conjunto'], group_keys=False).apply(remover_duplicados)


In [5]:
# Somar o valor_titulo das linhas de julho de 2025 em df_mes_filtrado
linhas_julho_2025 = df_mes_filtrado[
    (df_mes_filtrado['data_vencimento'].dt.year == 2025) &
    (df_mes_filtrado['data_vencimento'].dt.month == 7)
]
soma_valor_titulo_julho_2025 = linhas_julho_2025['valor_titulo'].sum()
print(f"Soma de valor_titulo das linhas de julho de 2025: {soma_valor_titulo_julho_2025}")


Soma de valor_titulo das linhas de julho de 2025: 47699516.599999994


In [ ]:
caminho_pasta = r'C:\Users\raphael.almeida\OneDrive - Grupo Unus\analise de dados - Arquivos em excel\Relatório de Inadimplência'
caminho_arquivo = os.path.join(caminho_pasta,'relatorio_faturas_mes.xlsx')
os.makedirs(caminho_pasta,exist_ok=True)
if os.path.exists(caminho_arquivo):
    os.remove(caminho_arquivo)
    print("Arquivo antigo removido, iniciando carregamento...")
df_mes.to_excel(caminho_arquivo, engine = 'openpyxl', index=False, sheet_name='faturas_mes')
print(f"Arquivo Excel salvo com sucesso em: {caminho_arquivo}")